<a href="https://colab.research.google.com/github/rrbauman/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graph

In [23]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-14 05:46:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-14 05:46:15 (9.65 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [26]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [27]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [18]:
df.count()

2166601

In [28]:
dropna_df = df.dropna()
dropna_df = dropna_df.distinct()
dropna_df.count()

2166461

In [20]:
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010D6B1QM7U|          1|            0|          0|   N|                Y|
|R100BI61LIKH2N|          5|            1|          2|   N|                Y|
|R100TPJCYDYGBU|          3|            1|          2|   N|                Y|
|R1018OO9U6UUWO|          5|            0|          0|   N|                Y|
|R102YIWZEIAXT9|          5|            2|          4|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [29]:
vine_total_votes_df= vine_df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_total_votes_df.show(5)







+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [30]:
from pyspark.sql.functions import col
vine_percents_df=vine_total_votes_df.withColumn('percent', col("helpful_votes")/ col("total_votes"))
vine_percents_df=vine_percents_df.filter('percent>=0.5')
vine_percents_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|0.9148936170212766|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|0.9347826086956522|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|0.8363636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|0.5714285714285714|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|             0.905|
| RZAE1OYOXEOAR|          5|           86|         94|   N|                Y|0.9148936170212766|
|R1TBFM7966BSCZ|          1|           33|         56|   N|                Y|0.5892857142857143|
|R3NSDYMPN0WV6M|          4|  

In [31]:
vine_Y_df=vine_percents_df.filter(col("vine").contains("Y"))
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R24PS9N276OXF4|          5|          241|        262|   Y|                N|0.9198473282442748|
|R3EPHK0JGQGE91|          5|           21|         28|   Y|                N|              0.75|
|R3T4LTJ189G0LT|          4|           37|         45|   Y|                N|0.8222222222222222|
| RKG407AVAZXBQ|          3|           22|         25|   Y|                N|              0.88|
| RNU8PK609WT6P|          4|          347|        362|   Y|                N|0.9585635359116023|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|0.7692307692307693|
|R23WY68NT94WKE|          5|          125|        143|   Y|                N|0.8741258741258742|
|R3JT907UUA1WUI|          4|  

In [32]:
vine_N_df=vine_percents_df.filter(col("vine").contains("N"))
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|0.9148936170212766|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|0.9347826086956522|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|0.8363636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|0.5714285714285714|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|             0.905|
| RZAE1OYOXEOAR|          5|           86|         94|   N|                Y|0.9148936170212766|
|R1TBFM7966BSCZ|          1|           33|         56|   N|                Y|0.5892857142857143|
|R3NSDYMPN0WV6M|          4|  

In [33]:
vine_Y_df.count()

94

In [34]:
vine_Y_df.filter(vine_Y_df.star_rating == 5).count()

48

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter pw')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge.cxoh7pryvfko.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter pw··········


In [ ]:
review_id_df.dropDuplicates().show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R33M4BPZPD8K08|   35949091|B00004TJL2|     780378241| 2003-07-02|
|R1N8AWNJHE8ZT2|   31748111|6301517644|     734783600| 2003-07-01|
| RB6UEQ1LO71O6|   38536412|B00005BJY7|     140970343| 2002-01-06|
|R2NONVGYN6IF6A|   51972364|B0000540FF|     849368135| 2001-02-16|
| RD5228M7GYAG5|   13913595|B0030K92MO|     765180606| 2015-05-06|
|R2J1L7JKBF3KLQ|   16571677|630301495X|     292337094| 2014-03-03|
|R1TKGKGGXXVGM0|   16307904|B005ZV54Z0|     201136089| 2013-08-02|
|R3T6LP8FAB00RU|   44402109|630018496X|     489430392| 2009-04-26|
|R2O1C0MRM28BSC|   52548425|6303042260|     566434820| 2006-09-25|
| RVLJ48RQJFWDC|   50011629|B00000ICEV|     480842390| 2001-01-16|
|R3REFKSZJ9KJAY|   49910802|6302772702|     752820095| 2000-06-29|
|R3AHPDY32V37GK|   51023085|6300182541|     816237993| 2000-04

In [ ]:
vine_df.dropna()
vine_df.dropDuplicates()


DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [ ]:
products_df.dropna()
products_df.dropDuplicates()

DataFrame[product_id: string, product_title: string]

In [ ]:
customers_df.dropna()
customers_df.dropDuplicates()

DataFrame[customer_id: int, customer_count: bigint]

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
print(products_df.distinct().count()) 

77020


In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
vine_total_votes_df= vine_df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_total_votes_df.show(5)






+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8U7JUF1JVKWE|          1|           36|         52|   N|                N|
| RGDM9MC0ZTUE9|          2|           26|         36|   N|                N|
|R1WF7SMH5P58H5|          1|            7|         22|   N|                N|
|R2DDJ8ZNI8O5PU|          5|           39|         41|   N|                Y|
| RN7S46SWDWU5N|          5|           61|         62|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
vine_helpful_votes_df = vine_total_votes_df.select([col("helpful_votes") / col("total_votes")].alias("votes_percent"))
# vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_helpful_votes_df.show()


NameError: ignored